# Environment 1 - Explicit feedback

This is a classical explicit feedback environment. We have users, items and the some of the corresponding ratings. We then need to be able to predict the other ratings.

**Imports**

In [1]:
import requests
import pandas as pd
import numpy as np

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


**Getting the data**

In [2]:
USER_ID = '9G08LOYFU88BJ8GHNRU3'
env = 'http://52.47.62.31/'

In [3]:
data = requests.get(url=env+'reset', params= {'user_id':USER_ID}).json()

nb_items = data['nb_items']
nb_users = data['nb_users']

item_history = data['item_history']
user_history = data['user_history']
rating_history = data['rating_history']

next_item = data['next_item']
next_user = data['next_user']

In [4]:
ratings = pd.DataFrame(data={'user_id':user_history, 'item_id':item_history, 'rating': rating_history})
ratings.head()

,user_id,item_id,rating
0,38,0,2
1,40,269,5
2,92,47,1
3,31,164,4
4,63,36,2


## Building the model

In [5]:
def build_model(embedding_size = 5):
    
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')

    user_embedding = Embedding(output_dim=embedding_size,
                               input_dim=nb_users + 1,
                               input_length=1,
                               name='user_embedding')(user_id_input)

    item_embedding = Embedding(output_dim=embedding_size,
                               input_dim=nb_items + 1,
                               input_length=1,
                               name='item_embedding')(item_id_input)

    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

    y = Dot(axes=1)([user_vecs, item_vecs])

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    
    return model

In [6]:
model = build_model()
model.summary()
model.compile(optimizer='adam', loss='MSE')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 5)         505         user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 5)         1505        item[0][0]                       
__________________________________________________________________________________________________
flatten_1 

## Training

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(
    [ratings['user_id'], ratings['item_id']], ratings['rating'],
    batch_size=64,
    epochs=20,
    validation_split=0.1,
    shuffle=True,
    callbacks=[early_stopping])

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 0s 46us/step - loss: 10.7330 - val_loss: 10.9953
Epoch 2/20
9000/9000 [==============================] - 0s 13us/step - loss: 10.6830 - val_loss: 10.8522
Epoch 3/20
9000/9000 [==============================] - 0s 11us/step - loss: 10.2506 - val_loss: 10.0021
Epoch 4/20
9000/9000 [==============================] - 0s 13us/step - loss: 8.9395 - val_loss: 8.3079
Epoch 5/20
9000/9000 [==============================] - 0s 11us/step - loss: 7.0956 - val_loss: 6.3970
Epoch 6/20
9000/9000 [==============================] - 0s 11us/step - loss: 5.2774 - val_loss: 4.6921
Epoch 7/20
9000/9000 [==============================] - 0s 11us/step - loss: 3.7658 - val_loss: 3.3500
Epoch 8/20
9000/9000 [==============================] - 0s 11us/step - loss: 2.6386 - val_loss: 2.3743
Epoch 9/20
9000/9000 [==============================] - 0s 16us/step - loss: 1.8598 - val_loss: 1.7207
Epoch 10/20
9000/90

## Testing and online training

In [8]:
nb_samples = 1000
mse = 0

users_list = []
ratings_list = []
items_list = []

for i in range(nb_samples):
    predicted_score = model.predict([[next_user], [next_item]])[0,0]
    
    r = requests.get(url=env + 'predict', params= {'user_id':USER_ID, 'predicted_score':predicted_score})
    true_rating = r.json()['rating']
    mse += (true_rating - predicted_score)**2
    
    users_list += [next_user]
    ratings_list += [true_rating]
    items_list += [next_item]
    
    if (i+1)%100 == 0:
        print("Episode {:d}/{:d}".format(i+1, nb_samples))
        model.fit([users_list,items_list], ratings_list, verbose=0, epochs=20)
        users_list = []
        ratings_list = []
        items_list = []
        
    next_item = r.json()['next_item']
    next_user = r.json()['next_user']
    
print('\nMSE: ', mse/nb_samples )

Episode 100/1000
Episode 200/1000
Episode 300/1000
Episode 400/1000
Episode 500/1000
Episode 600/1000
Episode 700/1000
Episode 800/1000
Episode 900/1000
Episode 1000/1000

MSE:  0.6588038064823045
